<a href="https://colab.research.google.com/github/moustafa-7/League-of-legends_Bigdata/blob/main/Bigdata_miniprojects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [28]:
import findspark
findspark.init()
import pyspark
import random

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LOL").getOrCreate()
sc = spark.sparkContext
d = spark.read.json("drive/MyDrive/small_matches.json", multiLine=True)

In [30]:
no_matches = d.count()

## 1. Champions win - pick - ban rates

In [31]:
def champions(row):
  c = []
  bans = []
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): team_number = 0
    else: team_number = 1
    c.append((p["championId"], row["teams"][team_number]["win"]))
  
  for t in row["teams"]:
    for b in t["bans"]:
      if(b["championId"] != None):
        c.append((b["championId"],"ban"))
      # else:
      #   c.append((b["championid"],"ban"))

  return c

In [32]:
champions = d.rdd.flatMap(champions)

In [33]:
import time
def lookup(champions):
  for c in champions.collect():
    print(c)
    time.sleep(0.1)

In [34]:
by_champion = champions.groupByKey().mapValues(list)

In [35]:
# for c, val in by_champion.collect():
#     print(c,val)
#     time.sleep(0.1)

In [36]:
def compute_champion_rates(val):
    ban_count = val.count("ban")
    win_count = val.count("Win")
    lose_count = val.count("Fail")
    total_played_games = win_count+lose_count
    if(total_played_games):
      return [ban_count/no_matches, win_count/total_played_games, lose_count/total_played_games]
    else:
      return [ban_count/no_matches, "never_played", "never_played"]

In [37]:
win_pick_ban_rates = by_champion.map(lambda x:(x[0], compute_champion_rates(x[1])))

In [ ]:
# for c in win_pick_ban_rates.collect():
#     print(c)
#     time.sleep(0.1)

## 2. Item Win - pick rates

In [86]:
def get_items(row):
  c = []
  bans = []
  for p in row["participants"]:
    for i in range(7):
      c.append((p["stats"]["item"+str(i)]["name"], p["stats"]["win"]))
  return c

In [87]:
items = d.rdd.flatMap(get_items).groupByKey().mapValues(list)

In [88]:

print(len(items.collect()))

190


In [94]:
def compute_item_rates(val):
  win_count = val.count(True)
  lose_count = val.count(False)
  total_played_games = win_count+lose_count
  return [win_count/total_played_games, len(val)/no_matches]

In [95]:
item_rates = items.map(lambda x:(x[0], compute_item_rates(x[1])))

In [ ]:
print("item       win_rate        pick_rate")
for item in item_rates.collect():
    print(item)
    time.sleep(0.01)